# Reduce the resolution of the water mask to match the population

In [1]:
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from affine import Affine
from numba import jit
from rasterio import features
from rasterio.crs import CRS
from rasterio.enums import Resampling
from rasterio.warp import reproject

from config import POP_DATA_SRC


def derez_water_mask(file_path, n_iters=1):
    """
    Load water mask and invert such that land is true and water is false
    De rex the water mask by performing 'or' of alternate rows
    and columns such that any point where one square is water (false)
    and the other square is land (true) the resulting point is considered as
    land, since the population may be non-zero.
    
    This is a bit fancier than just performing a decimated read 
    (https://mapbox.github.io/rasterio/topics/resampling.html)
    with the aim of avoiding dropping bits of the population
    """
    with rasterio.open(str(file_path)) as pop:
        print(pop.meta)
        pop_meta = pop.meta
        trns = pop.transform
#         population = pop.read(1, masked=True)
        population = pop.read_masks(1)

        population = population.astype(bool)
    for i in range(n_iters):
        # every other row
        population = population[::2, :] | population[1::2, :]
        # every other column
        population = population[:, ::2] | population[:, 1::2]
        # Output affine scaled by 2
        trns = Affine(trns.a * 2, trns.b, trns.c, trns.d, trns.e * 2, trns.f)
        
    return population, trns, pop_meta
    # Reduction to 1/4 of the original size already makes life much easier


def save_population_geotiff(population, trns, pop_meta):
    print('Saving')
    print(population.shape)
    population = population.astype(float)


    with rasterio.open(str(POP_DATA_SRC / 'nasa_grid' / _POPULATION_PATH_TEMPLATE.format(resolution=REZ_FIX)),
                       'w',
                       driver='GTiff',
                       height=population.shape[0],
                       width=population.shape[1],
                       count=1,
                       dtype=population.dtype,
                       crs=pop_meta['crs'],
                       transform=trns,
                       compress='lzw') as new_dataset:
        new_dataset.write(population, 1)

In [2]:
REZ_FIX = 'eightres'
_POPULATION_PATH_TEMPLATE = 'water_mask_{resolution}.tif'
input_file = POP_DATA_SRC / 'nasa_grid'/ 'gpw-v4-data-quality-indicators-water-mask' / 'gpw-v4-data-quality-indicators_water-mask.tif'
input_file = (POP_DATA_SRC / 'nasa_grid'/ 'count'/
              'gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals-2000' / 
              'gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals_2000.tif')

In [55]:
population, trns, pop_meta = derez_water_mask(input_file, n_iters=3)


{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -407649103380480.0, 'width': 43200, 'height': 17400, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.00833333333333339, 0.0, -180.0,
       0.0, -0.00833333333333339, 85.0000000000092)}


In [59]:
save_population_geotiff(population, trns, pop_meta)

Saving
(2175, 5400)
